In [3]:
import random

random.seed(42);

list_of_cakes = [random.choice([
    'sacher',
    'santiago',
    'san_marcos',
    'coca'])
    for _ in range(100)
]

In [6]:
list_of_cakes[0:10]

['sacher',
 'sacher',
 'san_marcos',
 'santiago',
 'santiago',
 'santiago',
 'sacher',
 'sacher',
 'coca',
 'sacher']

In [8]:
import pyspark

In [10]:
sc = SparkContext.getOrCreate();

In [11]:
sc

<SparkContext master=local[*] appName=PySparkShell>

Esto, como no tengo ningún clúster, lo va a usar como si fuera un clúster. Un clúster por núcleo del ordenador

In [12]:
my_first_rdd = sc.parallelize(list_of_cakes)
type(my_first_rdd)

pyspark.rdd.RDD

In [14]:
my_first_rdd.getNumPartitions()

12

Yo tengo 6 núcleos, ynme ha creado 12. No es directamente una relación 1 a 1 ni 1 a X. 

No sobreescribir usando my_first_rdd = my_first_rdd.map... Son inmutables. Aunque no acabo de pillarlo, porque sería sólo la referencia... A lo mejor van por valor

In [13]:
my_second_rdd = my_first_rdd.map(lambda x: x + 's')

In [15]:
my_second_rdd

PythonRDD[1] at RDD at PythonRDD.scala:53

Map es una transformación, no una acción. Esto es equivalente a lazy e eager operations en Linq

Esto ya haría que se ejecutara:

In [16]:
my_second_rdd.count()

100

Ejercicio: filtrar los pasteles que no empiezan por s

In [18]:
non_starting_with_s = my_first_rdd.filter(lambda x : x[0] != 's')

Take 3 sólo coge 3 y ya no sigue

In [19]:
non_starting_with_s.take(3)

['coca', 'coca', 'coca']

Collect es dame todo lo que tienen las particiones, que yo me lo guardo en mi RAM. Si el tamaño es considerable, se queda pillado, porque el master no va a tener suficiente ram. 

In [20]:
non_starting_with_s.collect()

['coca',
 'coca',
 'coca',
 'coca',
 'coca',
 'coca',
 'coca',
 'coca',
 'coca',
 'coca',
 'coca',
 'coca',
 'coca',
 'coca',
 'coca',
 'coca',
 'coca',
 'coca',
 'coca',
 'coca']

In [21]:
non_starting_with_s.count()

20

Lo normal es hacerlo estilo C#

In [23]:
my_first_rdd\
    .filter(lambda cake: cake[0] != 's')\
    .map(lambda x: x + 's')\
    .collect()

['cocas',
 'cocas',
 'cocas',
 'cocas',
 'cocas',
 'cocas',
 'cocas',
 'cocas',
 'cocas',
 'cocas',
 'cocas',
 'cocas',
 'cocas',
 'cocas',
 'cocas',
 'cocas',
 'cocas',
 'cocas',
 'cocas',
 'cocas']

## Qué pasa si no me cabe en RAM?  

Puedo escribir en disco, pero la cosa empezasrá a ir lento

(Ver solución)

Por otro lado, persist me permite guardar un estado por si em va a resultar interesamnte para operaciones posteriores. Esto prersiste en memoria por defecto o en disco si lo tiened así configurado y no cabe

## Pair RDD

SOn RDD que son un par de clave-valor (tanto clave como valor pueden ser objetos complejos)

In [27]:
pair_rdd = my_first_rdd.map(lambda cake: (cake,1)) # devuelves una tupla. La clave es el nombre. 
pair_rdd.collect()[0:10]

[('sacher', 1),
 ('sacher', 1),
 ('san_marcos', 1),
 ('santiago', 1),
 ('santiago', 1),
 ('santiago', 1),
 ('sacher', 1),
 ('sacher', 1),
 ('coca', 1),
 ('sacher', 1)]

In [28]:
grouped_rdd = pair_rdd.groupByKey()
grouped_rdd.collect()

[('sacher', <pyspark.resultiterable.ResultIterable at 0x11e2f0fd0>),
 ('san_marcos', <pyspark.resultiterable.ResultIterable at 0x11e2f06d0>),
 ('santiago', <pyspark.resultiterable.ResultIterable at 0x11e2f9bd0>),
 ('coca', <pyspark.resultiterable.ResultIterable at 0x11dfc03d0>)]

Esos iterables son listas de unos, en este caso

In [29]:
grouped_rdd.map(lambda tuple: (tuple[0],len(tuple[1]))).collect()

[('sacher', 27), ('san_marcos', 24), ('santiago', 29), ('coca', 20)]

Esto anterior me mapea de una lista de tuplas nomre-grupo a una lista de tuplas nombre-tamaño_del_grupo

NOTA: En el fondo, usando un map estamos haciendo una reducción

## Reduce by key 

In [31]:
pair_rdd.reduceByKey(lambda acc,x: acc + x).collect()

[('sacher', 27), ('san_marcos', 24), ('santiago', 29), ('coca', 20)]

## Apply word count to a file 

Queremos saber cuántas veces aparece cada palabra

In [33]:
!head data/shakespeare.txt

1609

THE SONNETS

by William Shakespeare



                     1
  From fairest creatures we desire increase,


In [34]:
shakespeare_lines = sc.textFile('data/shakespeare.txt')
shakespeare_lines.take(5) 

['1609', '', 'THE SONNETS', '', 'by William Shakespeare']

Vemos que el take coge las primeras 5 palabras. 

In [38]:
lower = shakespeare_lines.map(lambda word : word.lower())

In [39]:
lower.take(5)

['1609', '', 'the sonnets', '', 'by william shakespeare']

Mi solución

In [63]:
shakespeare_lines\
    .map(lambda line : line.lower())\
    .flatMap(lambda line : line.split(" "))\
    .map(lambda word: (word,1))\
    .groupByKey()\
    .map(lambda tuple: (tuple[0],len(tuple[1])))\
    .sortBy(lambda tuple : -tuple[1])\
    .take(5)

[('', 505702), ('the', 27267), ('and', 25340), ('i', 19540), ('to', 18656)]

Mejor usar reduceByKey

In [64]:
shakespeare_lines\
    .map(lambda line : line.lower())\
    .flatMap(lambda line : line.split(" "))\
    .map(lambda word: (word,1))\
    .reduceByKey(lambda acc,x: acc + x)\
    .sortBy(lambda tuple : -tuple[1])\
    .take(5)

[('', 505702), ('the', 27267), ('and', 25340), ('i', 19540), ('to', 18656)]

## Uso de dataframes en lugar de DDR (SparkSQL)

Es más sencillo y sigue estando optimizado por detrás. No te tienes que preocupar. Por debajo están los RDDs y tú te abstraes.

In [65]:
from pyspark.sql import SparkSession
session = SparkSession.builder.getOrCreate()

In [66]:
session

In [67]:
session.sparkContext.getConf().getAll()

[('spark.driver.host', 'juans-mbp'),
 ('spark.driver.port', '50370'),
 ('spark.sql.catalogImplementation', 'hive'),
 ('spark.rdd.compress', 'True'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.app.id', 'local-1571478677006'),
 ('spark.master', 'local[*]'),
 ('spark.executor.id', 'driver'),
 ('spark.submit.deployMode', 'client'),
 ('spark.app.name', 'PySparkShell'),
 ('spark.ui.showConsoleProgress', 'true')]

In [70]:
random.seed(42)
ids = range(12)
positions = [random.choice(['mecanic','sales','manager']) for id__ in ids]
positions

['manager',
 'mecanic',
 'mecanic',
 'manager',
 'sales',
 'mecanic',
 'mecanic',
 'mecanic',
 'manager',
 'mecanic',
 'manager',
 'manager']

In [72]:
rows = zip(ids,positions)
rows

In [73]:
df = session.createDataFrame(rows)
df

DataFrame[_1: bigint, _2: string]

La línea anterior tarda porque esta preparando las cosas, los RDD por debajo, etc.

In [74]:
df.take(5)

[Row(_1=0, _2='manager'),
 Row(_1=1, _2='mecanic'),
 Row(_1=2, _2='mecanic'),
 Row(_1=3, _2='manager'),
 Row(_1=4, _2='sales')]

In [76]:
myDf = session.createDataFrame(zip(ids,positions),schema=['id','position'])

In [77]:
myDf.show(5)

+---+--------+
| id|position|
+---+--------+
|  0| manager|
|  1| mecanic|
|  2| mecanic|
|  3| manager|
|  4|   sales|
+---+--------+
only showing top 5 rows

